In [61]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Read all data into one single dataframe
df_all_data = pd.read_csv('./dataset/storm_event_details_2010.csv')

for i in range(2011,2021):
    df_temp = pd.read_csv(f'./dataset/storm_event_details_{i}.csv')
    df_all_data = df_all_data.append(df_temp, ignore_index=True)


In [62]:
## Remove unused columns and format continuous columns

df_hur = df_all_data[df_all_data['EVENT_TYPE']=='Tornado']
df_hur = df_hur.drop(columns=['TOR_OTHER_WFO', 'END_YEARMONTH', 'EVENT_TYPE', 'END_DATE_TIME',
                                           'TOR_OTHER_CZ_STATE','TOR_OTHER_CZ_FIPS','TOR_OTHER_CZ_NAME','DATA_SOURCE','EPISODE_NARRATIVE',
                                            'EVENT_NARRATIVE','WFO','SOURCE','CZ_TIMEZONE','BEGIN_AZIMUTH','END_AZIMUTH','BEGIN_LAT',
                                            'END_LAT','BEGIN_LON','END_LON','STATE_FIPS','BEGIN_RANGE','END_RANGE','DAMAGE_CROPS',
                                            'BEGIN_TIME','END_TIME','BEGIN_LOCATION','END_LOCATION','FLOOD_CAUSE','MAGNITUDE_TYPE',
                                            'MAGNITUDE','CZ_FIPS','CZ_TYPE','CZ_NAME','CATEGORY'])
cols = ['INJURIES_INDIRECT', 'INJURIES_DIRECT', 'DEATHS_INDIRECT', 'DEATHS_DIRECT']
df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)
df_hur['TOR_AREA'] = df_hur['TOR_LENGTH']*df_hur['TOR_WIDTH']
df_hur = df_hur.drop(columns=['TOR_LENGTH', 'TOR_WIDTH'])



/tmp/ipykernel_2877/1111006650.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)


In [65]:
## Sort by priority variable and find data split percentages

df_hur = df_hur.sort_values('HARM_TOTAL', ascending=False)
df_hur.info()
df_hur = df_hur.loc[df_hur['TOR_F_SCALE']!='EFU']
print(df_hur['TOR_F_SCALE'].value_counts()/len(df_hur) * 100)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14555 entries, 100619 to 688592
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BEGIN_YEARMONTH  14555 non-null  int64  
 1   BEGIN_DAY        14555 non-null  int64  
 2   END_DAY          14555 non-null  int64  
 3   EPISODE_ID       14555 non-null  int64  
 4   EVENT_ID         14555 non-null  int64  
 5   STATE            14555 non-null  object 
 6   YEAR             14555 non-null  int64  
 7   MONTH_NAME       14555 non-null  object 
 8   BEGIN_DATE_TIME  14555 non-null  object 
 9   DAMAGE_PROPERTY  12575 non-null  object 
 10  TOR_F_SCALE      14555 non-null  object 
 11  HARM_TOTAL       14555 non-null  int64  
 12  TOR_AREA         14555 non-null  float64
dtypes: float64(1), int64(7), object(5)
memory usage: 1.6+ MB
EF0    50.697355
EF1    36.159395
EF2     9.742357
EF3     2.638269
EF4     0.680179
EF5     0.082446
Name: TOR_F_SCALE, dtype: float

In [69]:
## Split data into strata and sample proportinally

df_hur.groupby('TOR_F_SCALE', group_keys=False).apply(lambda x: x.sample(frac=0.1))

,BEGIN_YEARMONTH,BEGIN_DAY,END_DAY,EPISODE_ID,EVENT_ID,STATE,YEAR,MONTH_NAME,BEGIN_DATE_TIME,DAMAGE_PROPERTY,TOR_F_SCALE,HARM_TOTAL,TOR_AREA
238106,201305,30,30,74966,453305,OKLAHOMA,2013,May,30-MAY-13 14:08:00,0.00K,EF0,0,26.00
102774,201104,25,25,49476,290040,OKLAHOMA,2011,April,25-APR-11 16:18:00,5.00K,EF0,0,125.50
391790,201604,27,27,106675,640452,ILLINOIS,2016,April,27-APR-16 17:23:00,0.00K,EF0,0,123.50
399975,201605,21,21,105622,633635,KANSAS,2016,May,21-MAY-16 18:21:00,0.00K,EF0,0,3.25
382535,201505,8,8,96316,582213,OKLAHOMA,2015,May,08-MAY-15 17:16:00,0.00K,EF0,0,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
686281,202004,12,12,147593,887908,MISSISSIPPI,2020,April,12-APR-20 14:39:00,NaN,EF4,2,8645.00
75537,201104,27,27,49915,300457,TENNESSEE,2011,April,27-APR-11 19:34:00,23.00M,EF4,204,16320.00
671768,202004,13,13,147485,889068,SOUTH CAROLINA,2020,April,13-APR-20 05:10:00,NaN,EF4,65,30849.00
96657,201104,27,27,50516,302149,ALABAMA,2011,April,27-APR-11 16:05:00,20.00M,EF4,0,1944.80
